<a href="https://colab.research.google.com/github/AnkitaChakraborty1/Capstone_Adversarial_Robustness/blob/main/Keras_Goal_TransformationList_GreedyWordWeighted_Custom_Recipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##pip installs

In [1]:
pip install transformers

     |████████████████████████████████| 3.8 MB 4.2 MB/s 
     |████████████████████████████████| 6.5 MB 63.3 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 596 kB 70.2 MB/s 
     |████████████████████████████████| 895 kB 67.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip3 install textattack[tensorflow]

     |████████████████████████████████| 373 kB 4.2 MB/s 
     |████████████████████████████████| 101 kB 12.4 MB/s 
     |████████████████████████████████| 312 kB 88.0 MB/s 
     |████████████████████████████████| 322 kB 68.3 MB/s 
     |████████████████████████████████| 60 kB 9.0 MB/s 
     |████████████████████████████████| 769 kB 74.4 MB/s 
     |████████████████████████████████| 69 kB 7.1 MB/s 
     |████████████████████████████████| 454.3 MB 15 kB/s 
     |████████████████████████████████| 125 kB 99.0 MB/s 
     |████████████████████████████████| 4.9 MB 84.9 MB/s 
     |████████████████████████████████| 462 kB 92.7 MB/s 
     |████████████████████████████████| 1.2 MB 82.5 MB/s 
     |████████████████████████████████| 14.8 MB 82.7 MB/s 
     |████████████████████████████████| 4.0 MB 81.6 MB/s 
     |████████████████████████████████| 132 kB 92.3 MB/s 
     |████████████████████████████████| 4.9 MB 67.9 MB/s 
     |████████████████████████████████| 4.9 MB 76.8 MB/s 
     |████████████

In [3]:
pip install --upgrade numpy

     |████████████████████████████████| 15.7 MB 4.2 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.5.0 requires numpy~=1.19.2, but you have numpy 1.21.5 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [4]:
pip install folium==0.2.1

     |████████████████████████████████| 69 kB 3.2 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79808 sha256=0fcc6af4f98770774aedcb42ae7bf872b195715185b0cd3046efae80dc088593
  Stored in directory: /root/.cache/pip/wheels/9a/f0/3a/3f79a6914ff5affaf50cabad60c9f4d565283283c97f0bdccf
Successfully built folium
  Attempting uninstall: folium
    Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


## Imports

In [1]:
from textattack import AttackArgs, Attacker
from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import ModelWrapper
from textattack import Attack
from textattack.constraints.pre_transformation import (
        InputColumnModification,
        MaxModificationRate,
        RepeatModification,
        StopwordModification,
        MaxWordIndexModification
                )
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.goal_functions import UntargetedClassification,InputReduction,TargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.transformations import (
   
    WordSwapNeighboringCharacterSwap,
    WordSwapQWERTY,
    WordSwapWordNet,
    WordSwapRandomCharacterDeletion,
    WordSwapRandomCharacterInsertion,
    WordSwapRandomCharacterSubstitution,
    WordSwapHomoglyphSwap,
    WordSwapEmbedding,
    WordSwapHowNet,
    WordSwapInflections,
    WordDeletion,
    WordSwapGradientBased,
    WordSwapExtend,
    WordSwapContract,
    WordSwapChangeName,
    WordSwapChangeNumber,
    WordSwapChangeLocation,
    WordSwapRandomCharacterSubstitution,
    WordSwapMaskedLM,
    WordInsertionMaskedLM,
    WordMergeMaskedLM,
)
from textattack.constraints.overlap import LevenshteinEditDistance
#from textattack.attack_recipe import AttackRecipe

from abc import ABC, abstractmethod
from tqdm import tqdm # tqdm provides us a nice progress bar.
from textattack.loggers import CSVLogger # tracks a dataframe for us.
from IPython.core.display import display, HTML

In [2]:
from textattack.datasets import HuggingFaceDataset
from textattack.attacker import Attacker
from textattack import AttackArgs, Attacker
from textattack.datasets import HuggingFaceDataset
from textattack.goal_functions import TargetedClassification
import transformers
from textattack.models.wrappers import HuggingFaceModelWrapper
import torch

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/albert-base-v2-imdb")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/albert-base-v2-imdb")
# We wrap the model so it can be used by textattack
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

dataset = HuggingFaceDataset("rotten_tomatoes", None, "test", shuffle=True)
#dataset = HuggingFaceDataset("imdb", split="test")
#dataset = HuggingFaceDataset("glue", "sst2", "train")

Using custom data configuration default


textattack: Loading datasets dataset rotten_tomatoes, split test.


Dataset rotten_tomatoes_movie_review downloaded and prepared to /root/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/9198dbc50858df8bdb0d5f18ccaf33125800af96ad8434bc8b829918c987ee8a. Subsequent calls will reuse this data.


Creating the Keras Model

In [3]:
import tensorflow as tf
import tensorflow.keras
import numpy as np
from tensorflow.keras.utils import to_categorical
from textattack.models.wrappers import ModelWrapper
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import PWWSRen2019

import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout

from nltk.tokenize import word_tokenize, RegexpTokenizer

In [4]:
NUM_WORDS = 1000

(x_train_tokens, y_train), (x_test_tokens, y_test) = tf.keras.datasets.imdb.load_data(
    path="imdb.npz",
    num_words=NUM_WORDS,
    skip_top=0,
    maxlen=None,
    seed=113,
    start_char=1,
    oov_char=2,
    index_from=3
)

def transform(x):
  x_transform = []
  for i, word_indices in enumerate(x):
    BoW_array = np.zeros((NUM_WORDS,))
    for index in word_indices:
      if index < len(BoW_array):
        BoW_array[index] += 1
    x_transform.append(BoW_array)
  return np.array(x_transform)


index = int(0.9 * len(x_train_tokens))
x_train = transform(x_train_tokens)[:index]
x_test = transform(x_test_tokens)[index:]
y_train = np.array(y_train[:index])
y_test = np.array(y_test[index:])
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

vocabulary = tf.keras.datasets.imdb.get_word_index(
    path='imdb_word_index.json'
)

17465344/17464789 [==============================] - 1s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specif

1646592/1641221 [==============================] - 0s 0us/step


In [5]:
#Model Created with Keras
model = Sequential()
model.add(Dense(512, activation='relu', input_dim=NUM_WORDS))
model.add(Dropout(0.3))
model.add(Dense(100, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.00001)

model.compile(
 optimizer = opt,
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)


results = model.fit(
 x_train, y_train,
 epochs= 18,
 batch_size = 512,
 validation_data = (x_test, y_test)
)


print(results.history)

Epoch 1/18
44/44 [==============================] - 2s 7ms/step - loss: 0.8891 - accuracy: 0.5203 - val_loss: 0.6992 - val_accuracy: 0.5580
Epoch 2/18
44/44 [==============================] - 0s 3ms/step - loss: 0.8405 - accuracy: 0.5385 - val_loss: 0.6807 - val_accuracy: 0.6036
Epoch 3/18
44/44 [==============================] - 0s 3ms/step - loss: 0.8158 - accuracy: 0.5557 - val_loss: 0.6699 - val_accuracy: 0.6272
Epoch 4/18
44/44 [==============================] - 0s 3ms/step - loss: 0.7963 - accuracy: 0.5668 - val_loss: 0.6602 - val_accuracy: 0.6372
Epoch 5/18
44/44 [==============================] - 0s 3ms/step - loss: 0.7824 - accuracy: 0.5752 - val_loss: 0.6512 - val_accuracy: 0.6532
Epoch 6/18
44/44 [==============================] - 0s 3ms/step - loss: 0.7698 - accuracy: 0.5884 - val_loss: 0.6443 - val_accuracy: 0.6604
Epoch 7/18
44/44 [==============================] - 0s 3ms/step - loss: 0.7541 - accuracy: 0.5956 - val_loss: 0.6360 - val_accuracy: 0.6716
Epoch 8/18
44/44 [==

In [6]:
class CustomKerasModelWrapper(ModelWrapper):
    def __init__(self, model):
        self.model = model

    def __call__(self, text_input_list):

      x_transform = []
      for i, review in enumerate(text_input_list):
        tokens = [x.strip(",") for x in review.split()]
        BoW_array = np.zeros((NUM_WORDS,))
        for word in tokens:
          if word in vocabulary:
            if vocabulary[word] < len(BoW_array):
              BoW_array[vocabulary[word]] += 1
        x_transform.append(BoW_array)
      x_transform = np.array(x_transform)
      prediction = self.model.predict(x_transform)
      return prediction


CustomKerasModelWrapper(model)(["bad bad bad bad bad", "good good good good"])

array([[0.5035996 , 0.48540437],
       [0.5025748 , 0.5134921 ]], dtype=float32)

##Custom Recipes

In [7]:
class Recipe(Attack,ABC):
    @staticmethod
    def build(model_wrapper,transf,gf):
        if(transf == "WordSwapWordNet"):
            transformation = WordSwapWordNet()
        elif(transf == "WordSwapQWERTY"):
            transformation = WordSwapQWERTY()
        elif(transf == "WordSwapHomoglyphSwap"):
            transformation = WordSwapHomoglyphSwap()
        elif(transf == "WordSwapNeighboringCharacterSwap"):
            transformation = WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,)
        elif(transf == "WordSwapRandomCharacterInsertion"):
            transformation = WordSwapRandomCharacterInsertion(
                        random_one=True,
                        letters_to_insert=" ",
                        skip_first_char=True,
                        skip_last_char=True,
                    )
        elif(transf == "WordSwapRandomCharacterDeletion"):
             transformation = WordSwapRandomCharacterDeletion(
                        random_one=True, skip_first_char=True, skip_last_char=True
                    )
        elif(transf == "WordSwapEmbedding"):
             transformation = WordSwapEmbedding(max_candidates=5)
        elif(transf == "WordSwapHowNet"):
             transformation = WordSwapHowNet()
        elif(transf == "WordSwapInflections"):
             transformation = WordSwapInflections()
        elif(transf == "WordDeletion"):
             transformation = WordDeletion()
        elif(transf == "WordSwapGradientBased"):
             transformation = WordSwapGradientBased(model_wrapper, top_n=1)
        elif(transf == "WordSwapExtend"):
             transformation = WordSwapExtend()
        elif(transf == "WordSwapContract"):
             transformation = WordSwapContract()
        elif(transf == "WordSwapChangeName"):
             transformation = WordSwapChangeName()
        elif(transf == "WordSwapChangeNumber"):
             transformation = WordSwapChangeNumber()  
        elif(transf == "WordSwapChangeLocation"):
             transformation = WordSwapChangeLocation()
        elif(transf == "WordSwapRandomCharacterSubstitution"):
             transformation = WordSwapRandomCharacterSubstitution()
        elif(transf == "WordSwapMaskedLM"):
             transformation = WordSwapMaskedLM(method="bert-attack", max_candidates=48)
        elif(transf == "WordInsertionMaskedLM"):
             transformation = WordInsertionMaskedLM(
                    #masked_language_model=shared_masked_lm,
                    #tokenizer=shared_tokenizer,
                    max_candidates=50,
                    min_confidence=0.0,
                )
        elif(transf == "WordMergeMaskedLM"):
             transformation = WordMergeMaskedLM(
                    #masked_language_model=shared_masked_lm,
                    #tokenizer=shared_tokenizer,
                    max_candidates=50,
                    min_confidence=5e-3,
                )

        constraints = [RepeatModification(), StopwordModification()]
        if(gf == "TC"):
            goal_function = TargetedClassification(model_wrapper)
        elif(gf == "IR"):
            goal_function = InputReduction(model_wrapper)
        elif(gf == "UTC"):
            goal_function = UntargetedClassification(model_wrapper)
        # search over words based on a combination of their saliency score, and how efficient the WordSwap transform is
        search_method = GreedyWordSwapWIR("weighted-saliency")
        return Attack(goal_function, constraints, transformation, search_method)


Input Reduction - 'WordSwapWordNet','WordSwapHomoglyphSwap','WordSwapNeighboringCharacterSwap',
'WordSwapRandomCharacterDeletion','WordSwapEmbedding',
'WordSwapHowNet','WordDeletion','WordMergeMaskedLM',
'WordSwapRandomCharacterSubstitution'

In [8]:
transf_list = ['WordSwapWordNet','WordSwapHomoglyphSwap','WordSwapNeighboringCharacterSwap',
               'WordSwapRandomCharacterDeletion','WordSwapEmbedding',
               'WordSwapHowNet','WordDeletion','WordMergeMaskedLM',
               'WordSwapRandomCharacterSubstitution','WordSwapMaskedLM',
               'WordInsertionMaskedLM','WordMergeMaskedLM']
gf_list = ['IR','TC','UTC']
              
for gf in gf_list:         
    for transf in transf_list:
        print("Starting attack for-----------------",transf,gf)
        try:
            attack = Recipe.build(model_wrapper,transf,gf)
            attack_args = AttackArgs(num_examples=10,)
            attacker = Attacker(attack, dataset,attack_args)
            attack_results = attacker.attack_dataset()
        except ValueError:
            print("Not compatible")
            pass
        except KeyError:
            print("Key Error")
            pass

Starting attack for----------------- WordSwapWordNet IR


textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.input_reduction.InputReduction'>.
  0%|          | 0/10 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  InputReduction(
    (maximizable):  False
    (target_num_words):  1
  )
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:  10%|█         | 1/10 [00:04<00:41,  4.62s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (100%)]] --> [[[FAILED]]]

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .




...

[Succeeded / Failed / Skipped / Total] 0 / 5 / 2 / 7:  70%|███████   | 7/10 [02:44<01:10, 23.57s/it]

--------------------------------------------- Result 7 ---------------------------------------------
[[Positive (100%)]] --> [[[FAILED]]]

throws in enough clever and unexpected twists to make the formula feel fresh .




[Succeeded / Failed / Skipped / Total] 0 / 6 / 2 / 8:  80%|████████  | 8/10 [03:00<00:45, 22.53s/it]

--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (97%)]] --> [[[FAILED]]]

weighty and ponderous but every bit as filling as the treat of the title .




KeyboardInterrupt: ignored

Targeted and Untargeted Classification - 'WordSwapWordNet','WordSwapHomoglyphSwap','WordSwapNeighboringCharacterSwap',
               'WordSwapRandomCharacterDeletion','WordSwapEmbedding',
               'WordSwapHowNet','WordDeletion','WordMergeMaskedLM',
               'WordSwapRandomCharacterSubstitution',
               'WordInsertionMaskedLM'

In [10]:
transf_list = ['WordSwapWordNet','WordSwapHomoglyphSwap','WordSwapNeighboringCharacterSwap',
               'WordSwapRandomCharacterDeletion','WordSwapEmbedding',
               'WordSwapHowNet','WordDeletion','WordMergeMaskedLM',
               'WordSwapRandomCharacterSubstitution',
               'WordInsertionMaskedLM','WordMergeMaskedLM']
gf_list = ['UTC','TC']
              
for gf in gf_list:         
    for transf in transf_list:
        print("Starting attack for-----------------",transf,gf)
        try:
            attack = Recipe.build(model_wrapper,transf,gf)
            attack_args = AttackArgs(num_examples=10,)
            attacker = Attacker(attack, dataset,attack_args)
            attack_results = attacker.attack_dataset()
        except ValueError:
            print("Not compatible")
            pass
        except KeyError:
            print("Key Error")
            pass

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


  0%|          | 0/10 [00:00<?, ?it/s]

Starting attack for----------------- WordSwapWordNet UTC
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 





 10%|█         | 1/10 [00:02<00:23,  2.65s/it]

[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:02<00:23,  2.65s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (55%)]]

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 [[manages]] [[sweetness]] largely without stickiness .

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 [[contend]] [[sugariness]] largely without stickiness .






[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  20%|██        | 2/10 [00:03<00:12,  1.54s/it]

[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:  20%|██        | 2/10 [00:03<00:12,  1.55s/it]

[Succeeded / Failed / Skipped / Total] 2 / 0 / 1 / 3:  30%|███       | 3/10 [00:03<00:07,  1.04s/it]

...


Starting attack for----------------- WordInsertionMaskedLM TC


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.targeted_classification.TargetedClassification'>.


Not compatible
Starting attack for----------------- WordMergeMaskedLM TC


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.targeted_classification.TargetedClassification'>.


Not compatible


In [12]:
transf_list = [e]
gf_list = ['UTC','TC','IR']
              
for gf in gf_list:         
    for transf in transf_list:
        print("Starting attack for-----------------",transf,gf)
        try:
            attack = Recipe.build(model_wrapper,transf,gf)
            attack_args = AttackArgs(num_examples=10,)
            attacker = Attacker(attack, dataset,attack_args)
            attack_results = attacker.attack_dataset()
        except ValueError:
            print("Not compatible")
            pass
        except KeyError:
            print("Key Error")
            pass

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


  0%|          | 0/10 [00:00<?, ?it/s]

Starting attack for----------------- WordSwapExtend UTC
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapExtend
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 





 10%|█         | 1/10 [00:00<00:04,  2.17it/s]

[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:  10%|█         | 1/10 [00:00<00:04,  2.12it/s]

[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:  20%|██        | 2/10 [00:00<00:02,  3.39it/s]

[Succeeded / Failed / Skipped / Total] 0 / 2 / 0 / 2:  20%|██        | 2/10 [00:00<00:02,  3.36it/s]

[Succeeded / Failed / Skipped / Total] 0 / 2 / 1 / 3:  30%|███       | 3/10 [00:00<00:01,  4.79it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (100%)]] --> [[[FAILED]]]

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .


--------------------------------------------- Result 2 ---------------------------------------------
[[Positive (100%)]] --> [[[FAILED]]]

consistently clever and suspenseful .


--------------------------------------------- Result 3 ---------------------------------------------
[[Negative (85%)]] --> [[[SKIPPED]]]

it's like a " big chill " reunion of the baader-meinhof gang , only these guys are more harmless pranksters than political activists .






[Succeeded / Failed / Skipped / Total] 0 / 2 / 1 / 3:  40%|████      | 4/10 [00:01<00:01,  3.90it/s]

[Succeeded / Failed / Skipped / Total] 0 / 3 / 1 / 4:  40%|████      | 4/10 [00:01<00:01,  3.87it/s]

[Succeeded / Failed / Skipped / Total] 0 / 4 / 1 / 5:  50%|█████     | 5/10 [00:01<00:01,  4.22it/s]

[Succeeded / Failed / Skipped / Total] 0 / 4 / 1 / 5:  60%|██████    | 6/10 [00:01<00:00,  4.94it/s]

[Succeeded / Failed / Skipped / Total] 0 / 4 / 2 / 6:  60%|██████    | 6/10 [00:01<00:00,  4.93it/s]

...

--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (97%)]] --> [[[FAILED]]]

weighty and ponderous but every bit as filling as the treat of the title .






[Succeeded / Failed / Skipped / Total] 0 / 7 / 2 / 9:  90%|█████████ | 9/10 [00:02<00:00,  3.66it/s]

--------------------------------------------- Result 9 ---------------------------------------------
[[Positive (100%)]] --> [[[FAILED]]]

a real audience-pleaser that will strike a chord with anyone who's ever waited in a doctor's office , emergency room , hospital bed or insurance company office .






[Succeeded / Failed / Skipped / Total] 0 / 7 / 2 / 9: 100%|██████████| 10/10 [00:02<00:00,  3.73it/s]

[Succeeded / Failed / Skipped / Total] 0 / 8 / 2 / 10: 100%|██████████| 10/10 [00:02<00:00,  3.72it/s]

--------------------------------------------- Result 10 ---------------------------------------------
[[Positive (100%)]] --> [[[FAILED]]]

generates an enormous feeling of empathy for its characters .



+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 0     |
| Number of failed attacks:     | 8     |
| Number of skipped attacks:    | 2     |
| Original accuracy:            | 80.0% |
| Accuracy under attack:        | 80.0% |
| Attack success rate:          | 0.0%  |
| Average perturbed word %:     | nan%  |
| Average num. words per input: | 15.4  |
| Avg num queries:              | 14.5  |
+-------------------------------+-------+
